In [20]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!apt-get install g++ openjdk-8-jdk python-dev python3-dev
!pip3 install JPype1-py3
!pip install beautifulsoup4
!pip install sklearn
!pip install fake-useragent
!pip install pygame
!pip install --upgrade fake-useragent
!pip install webdriver-manager

Defaulting to user installation because normal site-packages is not writeable
패키지 목록을 읽는 중입니다... 완료0%
E: 잠금 파일 /var/lib/apt/lists/lock 파일을 열 수 없습니다 - open (13: Permission denied)
E: /var/lib/apt/lists/ 디렉터리를 잠글 수 없습니다
W: /var/cache/apt/pkgcache.bin 파일을 삭제하는데 문제가 있습니다 - RemoveCaches (13: Permission denied)
W: /var/cache/apt/srcpkgcache.bin 파일을 삭제하는데 문제가 있습니다 - RemoveCaches (13: Permission denied)
E: 잠금 파일 /var/lib/dpkg/lock-frontend 파일을 열 수 없습니다 - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
cp: cannot stat '/usr/lib/chromium-browser/chromedriver': No such file or directory
Defaulting to user installation because normal site-packages is not writeable
E: 잠금 파일 /var/lib/dpkg/lock-frontend 파일을 열 수 없습니다 - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
Defaulting to user installation because normal site-packages is not writeable


In [21]:
# -*- coding: UTF-8 -*-
import time
import os
import sys
import random
import requests
import json
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementClickInterceptedException, NoSuchElementException, StaleElementReferenceException, ElementNotInteractableException
from webdriver_manager.chrome import ChromeDriverManager
from PIL import Image
from pygame import mixer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from fake_useragent import UserAgent
from tqdm import tqdm
from datetime import datetime

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [37]:
# csv 파일이되 데이터 구분을 '|' 로 해둔 파일입니다. sep 지정을 안 하면 읽을 수 없습니다.
df = pd.read_csv('소상공인시장진흥공단_상가(상권)정보_제주_202212.csv', encoding='utf-8-sig')

# 음식점 데이터만 쓸 겁니다
df = df.loc[df['상권업종대분류명'] == '음식']  


# 다음과 같은 칼럼만 있으면 됩니다
df = df[['상호명', '상권업종중분류명', '상권업종소분류명', '표준산업분류명', '행정동명', '위도', '경도']]

# 칼럼명 단순화

df.columns = ['name',  # 상호명
              'cate_1',  # 중분류명
              'cate_2',  # 소분류명
              'cate_3',  # 표준산업분류명
              'dong',  # 행정동명
              'lon',  # 위도
              'lat'  # 경도
              ]
from sklearn.feature_extraction.text import CountVectorizer  # 피체 벡터화
from sklearn.metrics.pairwise import cosine_similarity  # 코사인 유사도


count_vect_category = CountVectorizer(min_df=0, ngram_range=(1,2))
place_category = count_vect_category.fit_transform(df['cate_mix'].values.astype( 'U' ))
place_simi_cate = cosine_similarity(place_category, place_category) 
place_simi_cate_sorted_ind = place_simi_cate.argsort()[:, ::-1]

options = webdriver.ChromeOptions()
#ua = UserAgent()
#user_agent = ua.random
#options.add_argument(f'user-agent={user_agent}')
options.add_argument('--headless') # Set Head-less
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-blink-features=AutomationControlled')
#driver = webdriver.Chrome('chromedriver', options=options)
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)


df['naver_keyword'] = df['dong'] + "%20" + df['name']  # "%20"는 띄어쓰기를 의미합니다.
df['naver_map_url'] = ''


# 본격적으로 가게 상세페이지의 URL을 가져옵시다

for i, keyword in enumerate(df['naver_keyword'].tolist()):
    print("이번에 찾을 키워드 :", i, f"/ {df.shape[0] -1} 행", keyword)
    try:
        naver_map_search_url = f"https://m.map.naver.com/search2/search.naver?query={keyword}&sm=hty&style=v5"
        
        driver.get(naver_map_search_url)
        time.sleep(3)
        df.iloc[i,-1] = driver.find_element(By.CSS_SELECTOR, "#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview").get_attribute('data-cid')
        # 네이버 지도 시스템은 data-cid에 url 파라미터를 저장해두고 있었습니다.
        # data-cid 번호를 뽑아두었다가 기본 url 템플릿에 넣어 최종적인 url을 완성하면 됩니다.
        
        #만약 검색 결과가 없다면?
    except Exception as e1:
        if "li:nth-child(1)" in str(e1):  # -> "child(1)이 없던데요?"
            try:
                df.iloc[i,-1] = driver.find_element(By.CSS_SELECTOR, "#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview").get_attribute('data-cid')
                time.sleep(1)
            except Exception as e2:
                print(e2)
                df.iloc[i,-1] = np.nan
                time.sleep(1)
        else:
            pass


driver.quit()

# URL이 수집되지 않은 데이터는 제거합니다.
df = df.loc[~df['naver_map_url'].isnull()]

#df['naver_map_url']은 각 가게별 Key값으로 사용 가능

# 이때 수집한 것은 완전한 URL이 아니라 URL에 들어갈 ID (data-cid 라는 코드명으로 저장된) 이므로, 온전한 URL로 만들어줍니다
df['가게정보'] = "https://pcmap.place.naver.com/restaurant/" + df['naver_map_url']+"/home"
df['리뷰링크'] = "https://pcmap.place.naver.com/restaurant/" + df['naver_map_url']+"/review/visitor"

df.to_csv('제주도_음식점_주소.csv', encoding = 'utf-8-sig', index=True)

,Tourist destination,dong,name
0,차이나타운,초량 1동,마루팥빙수단팥죽 본점
1,차이나타운,초량 1동,고민끝에여기
2,차이나타운,초량 1동,홍성방 본점
3,차이나타운,초량 1동,해신자연산횟집
4,차이나타운,초량 1동,시먼띵
...,...,...,...
115,부산아시아드조각광장,사직동,쭈꾸미본색 연산점
116,부산아시아드조각광장,사직동,아쿠마닭갈비포차
117,부산아시아드조각광장,사직동,부잣집 동태하우스 본점
118,부산아시아드조각광장,사직동,청솔물회


In [ ]:
# 가게별 '리뷰' 크롤링
# 파일 로드
result1 = pd.read_csv('제주도_음식점_주소.csv', encoding='utf-8-sig')

# 웹드라이버 옵션 설정
options = webdriver.ChromeOptions()
#ua = UserAgent()
#user_agent = ua.random
#options.add_argument(f'user-agent={user_agent}')
options.add_argument('--headless') # Set Head-less
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-blink-features=AutomationControlled')
#driver = webdriver.Chrome('chromedriver', options=options)


# 웹드라이버 초기화
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)

restaurant_name = []
restaurant_url = []
tourist_destination = []  
user_id = []
content = []
keywords = []
image_links = []  
author_names = []  # Add this line for author names
sub_counter = 0
counter = 0
sub_sub_counter = 0

# URL에 접속하여 크롤링 수행
for idx in range(len(result1['리뷰링크'])):
    counter = 0
    sub_counter = 0
    sub_sub_counter += 1
    driver.get(result1['리뷰링크'][idx])

    while True:
        try:
            driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
            driver.implicitly_wait(1)
            button = driver.find_element(By.CSS_SELECTOR, 'div.lfH3O a.fvwqf')
            button.click()
            counter += 1
            print(sub_sub_counter, "번째 가게의 더보기", counter, "번째 누르는 중")
            driver.implicitly_wait(1)
            driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
            driver.implicitly_wait(1)
            if counter == 100:
                print(sub_sub_counter, "번째 가게는 1000개 초과-")
                break
        except NoSuchElementException:
            print(sub_sub_counter, "번째 가게 리뷰 더보기 완료-")
            break
        except ElementClickInterceptedException:
            print(sub_sub_counter, "번째 가게는 막혀서 패스-")
            break
        except StaleElementReferenceException:
            print("StaleElementReferenceException가 발생했습니다. 요소를 다시 찾는 중입니다...")

    buttons = driver.find_elements(By.CSS_SELECTOR, ".P1zUJ.ZGKcF")
    for button in buttons:
        try:
            button.click()
            driver.implicitly_wait(1)
        except Exception as e:
            pass

    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    contents = soup.find_all('li', 'YeINN')

    for i in range(len(contents)):
        restaurant_name.append(result1['name'][idx])
        restaurant_url.append(result1['리뷰링크'][idx])
        tourist_destination.append(result1['Tourist destination'][idx])  # Append tourist destination

        # Extract author name
        author_name_div = contents[i].find('div', {'class': 'VYGLG'})
        if author_name_div:
            author_names.append(author_name_div.text)
        else:
            author_names.append(None)
        user_div = contents[i].find('div', attrs={'class': 'p24Ki'})
        if user_div:
            user_id.append(user_div.text)
        else:
            user_id.append(None)

        try:
            content_text = contents[i].find('span', attrs={'class': 'zPfVt'}).text.replace('\n', ' ')
        except AttributeError:
            content_text = None
        content.append(content_text)

        gyagi_div = contents[i].find('div', attrs={'class': 'gyAGI'})
        if gyagi_div:
            keyword_elements = gyagi_div.find_all('span', attrs={'class': 'P1zUJ'})
            keyword_texts = [keyword_element.text for keyword_element in keyword_elements]
            keywords.append(keyword_texts if keyword_texts else None)
        else:
            keywords.append(None)

        # Extracting image links
        image_div = contents[i].find('div', {'class': 'ODalI'})
        if image_div:
            style_attr = image_div.find('div', {'class': 'K0PDV'})['style']
            image_url = re.search(r'url\("(.+?)"\)', style_attr)
            if image_url:
                image_links.append(image_url.group(1))
            else:
                image_links.append(None)
        else:
            image_links.append(None)
    sub_counter += 1

    print(sub_sub_counter, "번째 가게까지", len(user_id), "개 리뷰 수집 완료.")

# 데이터프레임 생성
naver_review = pd.DataFrame({
    'Name': restaurant_name,
    'URL': restaurant_url,
    'Tourist_destination': tourist_destination,
    'Author_Name': author_names,  # Add this line for author names
    'ID': user_id,
    'Content': content,
    'Keywords': keywords,
    'Image Links': image_links
})

# CSV 파일로 저장
naver_review.to_csv('제주도_음식점_리뷰_크롤링_완료.csv', encoding='utf-8-sig', index=True)

# 데이터프레임 출력
naver_review


In [48]:
# '가게 정보' 크롤링
# 파일 로드
result1 = pd.read_csv('가게소개용.csv', encoding='cp949')

# 웹드라이버 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-blink-features=AutomationControlled')

# 웹드라이버 초기화
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)

description = []
store_type = []  # List to store store type text
review_counts = []  # List to store review counts
sub_counter = 0
counter = 0
sub_sub_counter = 0

# URL에 접속하여 크롤링 수행
for idx in range(len(result1['가게정보'])):
    driver.get(result1['가게정보'][idx])
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'body')))

    # 더보기 버튼이 있다면 클릭
    try:
        more_button = driver.find_element_by_css_selector('a.xHaT3 span.rvCSr')
        more_button.click()
        # 대기
        WebDriverWait(driver, 3)
    except Exception:
        pass  # 더보기 버튼이 없는 경우, 그냥 넘어감

    # 페이지 로딩이 완료된 후 HTML 소스 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')

    # span 태그 내의 텍스트를 추출 (Description)
    try:
        desc = soup.find('a', {'class': 'xHaT3'}).find('span', {'class': 'zPfVt'}).text
        description.append(desc)
    except AttributeError:
        description.append("Description not found")

    # Extracting store type
    try:
        store = soup.find('span', {'class': 'DJJvD'}).text
        store_type.append(store)
    except AttributeError:
        store_type.append("Store type not found")

    # Extracting review counts
    try:
        review_count = soup.find('div', {'class': 'dAsGb'}).find('a', role="button").find('em').text
        review_counts.append(review_count)
    except AttributeError:
        review_counts.append("Review counts not found")
    
    print(f"Crawling completed for store {idx + 1}")

# 데이터프레임에 설명, 상점 유형, 리뷰 개수 컬럼 추가
result1['Description'] = description
result1['Store_Type'] = store_type
result1['Review_Counts'] = review_counts  # Adding review counts to DataFrame

# 데이터프레임으로 저장
result1.to_csv('가게정보크롤링.csv', encoding='utf-8-sig', index=False)

# 웹드라이버 종료
driver.quit()


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST chromedriver version for 87.0.4280 google-chrome
[WDM] - Driver [/home/nampromotion/.wdm/drivers/chromedriver/linux64/87.0.4280.88/chromedriver] found in cache


Crawling completed for store 1
Crawling completed for store 2
Crawling completed for store 3
Crawling completed for store 4
Crawling completed for store 5
Crawling completed for store 6
Crawling completed for store 7
Crawling completed for store 8
Crawling completed for store 9
Crawling completed for store 10
Crawling completed for store 11
Crawling completed for store 12
Crawling completed for store 13
Crawling completed for store 14
Crawling completed for store 15
Crawling completed for store 16
Crawling completed for store 17
Crawling completed for store 18
Crawling completed for store 19
Crawling completed for store 20
Crawling completed for store 21
Crawling completed for store 22
Crawling completed for store 23
Crawling completed for store 24
Crawling completed for store 25
Crawling completed for store 26
Crawling completed for store 27
Crawling completed for store 28
Crawling completed for store 29
Crawling completed for store 30
Crawling completed for store 31
Crawling complete

Crawling completed for store 253
Crawling completed for store 254
Crawling completed for store 255
Crawling completed for store 256
Crawling completed for store 257
Crawling completed for store 258
Crawling completed for store 259
Crawling completed for store 260
Crawling completed for store 261
Crawling completed for store 262
Crawling completed for store 263
Crawling completed for store 264
Crawling completed for store 265
Crawling completed for store 266
Crawling completed for store 267
Crawling completed for store 268
Crawling completed for store 269
Crawling completed for store 270
Crawling completed for store 271
Crawling completed for store 272
Crawling completed for store 273
Crawling completed for store 274
Crawling completed for store 275
Crawling completed for store 276
Crawling completed for store 277
Crawling completed for store 278
Crawling completed for store 279
Crawling completed for store 280
Crawling completed for store 281
Crawling completed for store 282
Crawling c

Crawling completed for store 503
Crawling completed for store 504
Crawling completed for store 505
Crawling completed for store 506
Crawling completed for store 507
Crawling completed for store 508
Crawling completed for store 509
Crawling completed for store 510
Crawling completed for store 511
Crawling completed for store 512
Crawling completed for store 513
Crawling completed for store 514
Crawling completed for store 515
Crawling completed for store 516
Crawling completed for store 517
Crawling completed for store 518
Crawling completed for store 519
Crawling completed for store 520
Crawling completed for store 521
Crawling completed for store 522
Crawling completed for store 523
Crawling completed for store 524
Crawling completed for store 525
Crawling completed for store 526
Crawling completed for store 527
Crawling completed for store 528
Crawling completed for store 529
Crawling completed for store 530
Crawling completed for store 531
Crawling completed for store 532
Crawling c